# MVSJ Templates

This notebook demonstrates how to create and use templates in MolViewSpec.
Templates allow you to define a structure once with placeholder values,
then fill in the placeholders later to create multiple similar visualizations.

In [ ]:
import { createBuilder,findRef, State, Node, molstarNotebook} from "../../molviewspec-ts/mod.ts";

## Creating a Template

Create a template with a placeholder URL and a reference to identify the node.

In [ ]:
const builder = createBuilder();

// Create a template with a placeholder URL and a reference
builder
  .download({ url: "<placeholder>", ref: "download" })
  .parse({ format: "mmcif" })
  .modelStructure()
  .component({ selector: "all" })
  .representation({ type: "cartoon" })
  .color({ color: "blue" });

// Get the template as JSON string
const templateState = builder.getState();
const templateJson = JSON.stringify(templateState, null, 2);

// console.log("Template created:");
// console.log(templateJson);

// In a real scenario, you would save this to a file:
// await Deno.writeTextFile("./path/to/template.mvsj", templateJson);
// await molstarNotebook(templateState)

## Using the Template

Load the template and fill in the placeholder with an actual URL.

In [ ]:
// Load the template from JSON
const loadedTemplate: State = JSON.parse(templateJson);

// Find the node with the reference "download"
const downloadNode = findRef(loadedTemplate.root, "download");

if (downloadNode && downloadNode.params) {
  // Replace the placeholder with an actual URL
  downloadNode.params["url"] = "https://files.wwpdb.org/download/1cbs.cif";
  
  console.log("Template filled with 1cbs:");
  console.log(JSON.stringify(loadedTemplate, null, 2));
} else {
  console.error("Download node not found!");
}

## Creating Multiple Instances from Template

Use the same template to create visualizations for different PDB structures.

In [ ]:
// Helper function to create a state from template
function fillTemplate(templateJson: string, pdbId: string, color: string): State {
  const state: State = JSON.parse(templateJson);
  
  // Find and update the download node
  const downloadNode = findRef(state.root, "download");
  if (downloadNode && downloadNode.params) {
    downloadNode.params["url"] = `https://files.wwpdb.org/download/${pdbId}.cif`;
  }
  
  // Find and update the color node (assuming it's deep in the tree)
  // For simplicity, we'll do a recursive search
  function updateColor(node: Node, newColor: string): void {
    if (node.kind === "color" && node.params) {
      node.params["color"] = newColor;
    }
    if (node.children) {
      for (const child of node.children) {
        updateColor(child, newColor);
      }
    }
  }
  
  updateColor(state.root, color);
  
  // Update metadata
  state.metadata.title = `PDB ${pdbId}`;
  state.metadata.description = `Structure visualization of ${pdbId}`;
  
  return state;
}

// Create multiple instances
const pdbIds = ["1cbs", "1tqn", "2hhb"];
const colors = ["blue", "red", "green"];

for (let i = 0; i < pdbIds.length; i++) {
  const instance = fillTemplate(templateJson, pdbIds[i], colors[i]);
  console.log(`\\nInstance ${i + 1} (${pdbIds[i]}): `);
  console.log(JSON.stringify(instance, null, 2));
}

## Advanced Template with Multiple References

Create a more complex template with multiple customizable elements.

In [ ]:
const advancedBuilder = createBuilder();

const struct = advancedBuilder
  .download({ url: "&lt;placeholder&gt;", ref: "download" })
  .parse({ format: "mmcif" })
  .modelStructure();

// Polymer representation with reference
struct
  .component({ selector: "polymer" })
  .representation({ type: "cartoon", ref: "polymer_rep" })
  .color({ color: "&lt;polymer_color&gt;", ref: "polymer_color" });

// Ligand representation with reference
struct
  .component({ selector: "ligand" })
  .representation({ type: "ball_and_stick", ref: "ligand_rep" })
  .color({ color: "&lt;ligand_color&gt;", ref: "ligand_color" });

const advancedTemplateState = advancedBuilder.getState();
const advancedTemplateJson = JSON.stringify(advancedTemplateState, null, 2);

console.log("Advanced template:");
console.log(advancedTemplateJson);

In [ ]:
// Helper function to fill advanced template
function fillAdvancedTemplate(
  templateJson: string,
  pdbId: string,
  polymerColor: string,
  ligandColor: string
): State {
  const state: State = JSON.parse(templateJson);
  
  // Update download URL
  const downloadNode = findRef(state.root, "download");
  if (downloadNode && downloadNode.params) {
    downloadNode.params["url"] = `https://files.wwpdb.org/download/${pdbId}.cif`;
  }
  
  // Update polymer color
  const polymerColorNode = findRef(state.root, "polymer_color");
  if (polymerColorNode && polymerColorNode.params) {
    polymerColorNode.params["color"] = polymerColor;
  }
  
  // Update ligand color
  const ligandColorNode = findRef(state.root, "ligand_color");
  if (ligandColorNode && ligandColorNode.params) {
    ligandColorNode.params["color"] = ligandColor;
  }
  
  return state;
}

// Use the advanced template
const customizedState = fillAdvancedTemplate(
  advancedTemplateJson,
  "1cbs",
  "cyan",
  "magenta"
);

console.log("Customized state:");
console.log(JSON.stringify(customizedState, null, 2));

await molstarNotebook(customizedState)